In [ ]:
from utils import *

# Crop Production

In [92]:
# load FAO data
df_production_all = pd.read_csv(path_data / "FAO/production/Production_Crops_Livestock_E_All_Data_NOFLAG.csv", encoding_errors="ignore")\
    .query("Element == 'Production'")\
    .query("`Area Code`<5000")\
    .query("Unit != '1000 No'")

col_sel = ["Area Code", "Area Code (M49)", "Area", "Item Code", "Item Code (CPC)", "Item", "Element Code", "Element", "Unit"]
col_sel = [*col_sel, *[f"Y{year}" for year in range(2000, 2023)]]

df_production_all = df_production_all[col_sel]
sel_idx = df_production_all[[f"Y{year}" for year in range(2000, 2023)]].dropna(how="all").index
df_production_all = df_production_all.loc[sel_idx].reset_index(drop=True)

In [3]:
# Data sepration for countries that became independent after the year 2000
def seperate_data(df_production_all=df_production_all, former='Sudan (former)', new_1='South Sudan', new_2='Sudan'):
    
    sel_lst = [former, new_1, new_2]
    df_sudan = df_production_all.query("Area in @sel_lst")
    df_sd_sum = df_sudan.groupby(["Item",])[[f"Y{year}" for year in range(2000, 2023)]].sum()

    ratio_ssd = (df_sudan.query("Area == @new_1").sort_values("Item").set_index("Item")[[f"Y{year}" for year in range(2000, 2023)]] / df_sd_sum).mean(axis=1)
    df_sd_fill_ssd = df_sd_sum.apply(lambda x: x * ratio_ssd)
    df_sd_fill_sd = df_sd_sum.apply(lambda x: x * (1 - ratio_ssd))

    fill_dic = {new_1: df_sd_fill_ssd, new_2: df_sd_fill_sd}
    df_sudan_fillna = df_sudan.query("Area != @former").copy()
    for year in range(2000, 2023):
        for idx in df_sudan_fillna.index:
            if np.isnan(df_sudan_fillna.loc[idx, f"Y{year}"]):
                area_ = df_sudan_fillna.loc[idx, "Area"]
                item_ = df_sudan_fillna.loc[idx, "Item"]
                df_fill = fill_dic[area_]
                if item_ in df_fill.index:
                    df_sudan_fillna.loc[idx, f"Y{year}"] = df_fill.loc[item_, f"Y{year}"].round(1)
    return df_sudan, df_sudan_fillna

In [4]:
# Sudan (former) --> South Sudan / Sudan
df_sudan, df_sudan_fillna = seperate_data(df_production_all=df_production_all, former='Sudan (former)', new_1='South Sudan', new_2='Sudan')
# Serbia and Montenegro --> Serbia / Montenegro
df_S_M, df_S_M_fillna = seperate_data(df_production_all=df_production_all, former='Serbia and Montenegro', new_1='Serbia', new_2='Montenegro')

In [5]:
df_prod_r1 = pd.concat([
    df_production_all.drop(df_sudan.index).drop(df_S_M.index),
    df_sudan_fillna, df_S_M_fillna
]).reset_index(drop=True)
df_prod_r1.columns =  df_prod_r1.columns.map(lambda x: int(x[1:]) if x.startswith("Y2") else x)

In [ ]:
# Standardizing country name spellings and regions from different sources
dic_name = {"Afghanistan": "Afghanistan", "Albania": "Albania", "Algeria": "Algeria", "Angola": "Angola", "Antigua and Barbuda": "Antigua and Barbuda", "Argentina": "Argentina", "Armenia": "Armenia", "Australia": "Australia", "Austria": "Austria", "Azerbaijan": "Azerbaijan", "Bahamas": "Bahamas", "Bahrain": "Bahrain", "Bangladesh": "Bangladesh", "Barbados": "Barbados", "Belarus": "Belarus", "Belgium": "Belgium", "Belize": "Belize", "Benin": "Benin", "Bhutan": "Bhutan", "Bosnia and Herzegovina": "Bosnia and Herzegovina", "Botswana": "Botswana", "Brazil": "Brazil", "Brunei Darussalam": "Brunei Darussalam", "Bulgaria": "Bulgaria", "Burkina Faso": "Burkina Faso", "Burundi": "Burundi", "Cambodia": "Cambodia", "Cameroon": "Cameroon", "Canada": "Canada", "Central African Republic": "Central African Republic", "Chad": "Chad", "Chile": "Chile", "China": "China", "Colombia": "Colombia", "Comoros": "Comoros", "Cook Islands": "Cook Islands", "Costa Rica": "Costa Rica", "Croatia": "Croatia", "Cuba": "Cuba", "Cyprus": "Cyprus", "Democratic Republic of the Congo": "Democratic Republic of the Congo", "Denmark": "Denmark", "Djibouti": "Djibouti", "Dominica": "Dominica", "Dominican Republic": "Dominican Republic", "Ecuador": "Ecuador", "Egypt": "Egypt", "El Salvador": "El Salvador", "Equatorial Guinea": "Equatorial Guinea", "Eritrea": "Eritrea", "Estonia": "Estonia", "Ethiopia": "Ethiopia", "Fiji": "Fiji", "Finland": "Finland", "France": "France", "French Polynesia": "French Polynesia", "Gabon": "Gabon", "Georgia": "Georgia", "Germany": "Germany", "Ghana": "Ghana", "Greece": "Greece", "Grenada": "Grenada", "Guatemala": "Guatemala", "Guinea": "Guinea", "Guinea-Bissau": "Guinea-Bissau", "Guyana": "Guyana", "Haiti": "Haiti", "Honduras": "Honduras", "Hungary": "Hungary", "Iceland": "Iceland", "India": "India", "Indonesia": "Indonesia", "Iraq": "Iraq", "Ireland": "Ireland", "Israel": "Israel", "Italy": "Italy", "Jamaica": "Jamaica", "Japan": "Japan", "Jordan": "Jordan", "Kazakhstan": "Kazakhstan", "Kenya": "Kenya", "Kiribati": "Kiribati", "Kuwait": "Kuwait", "Kyrgyzstan": "Kyrgyzstan", "Latvia": "Latvia", "Lebanon": "Lebanon", "Lesotho": "Lesotho", "Liberia": "Liberia", "Libya": "Libya", "Lithuania": "Lithuania", "Luxembourg": "Luxembourg", "Madagascar": "Madagascar", "Malawi": "Malawi", "Malaysia": "Malaysia", "Maldives": "Maldives", "Mali": "Mali", "Malta": "Malta", "Marshall Islands": "Marshall Islands", "Mauritania": "Mauritania", "Mauritius": "Mauritius", "Mexico": "Mexico", "Mongolia": "Mongolia", "Montenegro": "Montenegro", "Morocco": "Morocco", "Mozambique": "Mozambique", "Myanmar": "Myanmar", "Namibia": "Namibia", "Nauru": "Nauru", "Nepal": "Nepal", "New Caledonia": "New Caledonia", "New Zealand": "New Zealand", "Nicaragua": "Nicaragua", "Niger": "Niger", "Nigeria": "Nigeria", "Niue": "Niue", "Norway": "Norway", "Oman": "Oman", "Pakistan": "Pakistan", "Palestine": "Palestine", "Panama": "Panama", "Papua New Guinea": "Papua New Guinea", "Paraguay": "Paraguay", "Peru": "Peru", "Philippines": "Philippines", "Poland": "Poland", "Portugal": "Portugal", "Puerto Rico": "Puerto Rico", "Qatar": "Qatar", "Republic of Korea": "Republic of Korea", "Romania": "Romania", "Russian Federation": "Russian Federation", "Rwanda": "Rwanda", "Saint Kitts and Nevis": "Saint Kitts and Nevis", "Saint Lucia": "Saint Lucia", "Saint Vincent and the Grenadines": "Saint Vincent and the Grenadines", "Samoa": "Samoa", "Saudi Arabia": "Saudi Arabia", "Senegal": "Senegal", "Serbia": "Serbia", "Seychelles": "Seychelles", "Sierra Leone": "Sierra Leone", "Singapore": "Singapore", "Slovakia": "Slovakia", "Slovenia": "Slovenia", "Solomon Islands": "Solomon Islands", "Somalia": "Somalia", "South Africa": "South Africa", "South Sudan": "South Sudan", "Spain": "Spain", "Sri Lanka": "Sri Lanka", "Sudan": "Sudan", "Suriname": "Suriname", "Sweden": "Sweden", "Switzerland": "Switzerland", "Tajikistan": "Tajikistan", "Thailand": "Thailand", "Timor-Leste": "Timor-Leste", "Togo": "Togo", "Tonga": "Tonga", "Trinidad and Tobago": "Trinidad and Tobago", "Tunisia": "Tunisia", "Turkmenistan": "Turkmenistan", "Tuvalu": "Tuvalu", "Uganda": "Uganda", "Ukraine": "Ukraine", "United Arab Emirates": "United Arab Emirates", "Uruguay": "Uruguay", "Uzbekistan": "Uzbekistan", "Vanuatu": "Vanuatu", "Yemen": "Yemen", "Zambia": "Zambia", "Zimbabwe": "Zimbabwe", "Bolivia (Plurinational State of)": "Bolivia", "Cabo Verde": "Cape Verde", "China, Hong Kong SAR": "Hong Kong", "China, Macao SAR": "Macao", "China, mainland": "China", "China, Taiwan Province of": "Taiwan", "Congo": "Republic of Congo", "Cte d'Ivoire": "Cte d'Ivoire", "Czechia": "Czech Republic", "Democratic People's Republic of Korea": "Dem. Rep. Korea", "Eswatini": "Swaziland", "Faroe Islands": "Faeroe Islands", "French Guiana": "French Guiana", "Gambia": "The Gambia", "Guadeloupe": "Guadeloupe", "Iran (Islamic Republic of)": "Iran", "Lao People's Democratic Republic": "Lao PDR", "Martinique": "Martinique", "Micronesia (Federated States of)": "Federated States of Micronesia", "Netherlands (Kingdom of the)": "Netherlands", "North Macedonia": "Macedonia", "Republic of Moldova": "Moldova", "Runion": "Réunion", "Sao Tome and Principe": "S?o Tomé and Principe", "Serbia": "Serbia", "Montenegro": "Montenegro","Sudan": "Sudan","south Sudan": "South Sudan","Syrian Arab Republic": "Syria", "Tokelau": "New Zealand", "Trkiye": "Turkey", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "United Republic of Tanzania": "Tanzania", "United States of America": "United States", "Venezuela (Bolivarian Republic of)": "Venezuela", "Viet Nam": "Vietnam", }
dic_region = {"Northern Africa and Western Asia": "N. Africa W. Asia", "Sub-Saharan Africa": "Sub-Saharan Africa", "Central Asia and Russian Federation": "C. Asia", "Eastern Asia": "E. Asia", "Southern Asia": "S. Asia", "Southeastern Asia": "S. Asia", "Northern America": "N. America", "Latin America and the Caribbean": "Latin America", "Western Europe": "W. Europe", "Eastern and South-Eastern Europe": "E. Europe", "Oceania and Australia": "Oceania",}

gdf_world = gpd.read_file(path_data / "vector/globalmap_revise/map.shp").clip(box(-180, -60, 180, 85))
gdf_world["regi_short"] = gdf_world["regi_pnas"].map(dic_region)
dict_region = gdf_world.set_index("name_long")["regi_short"].to_dict()
df_prod_r1["region"] = df_prod_r1["name_long"].map(dict_region)

In [15]:
from crop_product_factor import get_df_crop_factor
df_crop_factor = get_df_crop_factor()
sel_items = set(df_crop_factor["Item"])

In [16]:
df_harv_f = pd.read_excel(path_data / "FAO/yield and stock/crop_factor.xlsx", sheet_name="harvest factor")
df_harv_f = df_harv_f.melt(id_vars="harvest factor").rename(columns={"harvest factor": "harv_f_type", "variable": "region", "value": "hf"})
df_rec_r = pd.read_excel(path_data / "FAO/yield and stock/crop_factor.xlsx", sheet_name="recovery rate")
df_rec_r = df_rec_r.melt(id_vars="recovery rate").rename(columns={"recovery rate": "rec_r_type", "variable": "region", "value": "rr"})

In [ ]:
df_prod_r2 = df_prod_r1.query("Item in @sel_items")\
    .assign(harv_f_type=lambda _df: _df["Item"].map(df_crop_factor.set_index("Item")["harvest factor"].to_dict()))\
    .assign(rec_r_type=lambda _df: _df["Item"].map(df_crop_factor.set_index("Item")["recovery rate"].to_dict()))\
    .assign(moisture=lambda _df: _df["Item"].map(df_crop_factor.set_index("Item")["moisture"].to_dict()))
df_prod_r2 = df_prod_r2.merge(df_harv_f, on=["region", "harv_f_type"], how="left")
df_prod_r2 = df_prod_r2.merge(df_rec_r, on=["region", "rec_r_type"], how="left")
df_prod_r2 = df_prod_r2.drop(columns=["Area Code", "Area Code (M49)", "Item Code", "Item Code (CPC)", "Element", "Element Code", 'harv_f_type', 'rec_r_type'])

In [ ]:
# convert to dry matter (g)
df_prod_DMg = df_prod_r2.copy().drop(columns=["Unit"])
# DM
df_prod_DMg.loc[:, [*np.arange(2000, 2023)]] = df_prod_DMg.loc[:, [*np.arange(2000, 2023)]].apply(lambda x: x*(1-df_prod_DMg["moisture"]/100))
# t --> g
df_prod_DMg.loc[:, [*np.arange(2000, 2023)]] = df_prod_DMg.loc[:, [*np.arange(2000, 2023)]] * 1000000 
df_prod_DMg = df_prod_DMg.drop(columns=["moisture"])

In [ ]:
# calculate residue
df_residue_DMg = df_prod_DMg.copy()
df_residue_DMg.loc[:, [*np.arange(2000, 2023)]] = df_residue_DMg.loc[:, [*np.arange(2000, 2023)]].apply(lambda x: x * df_residue_DMg["hf"])

In [23]:
# convert dry matter to C
df_prod_harvC = df_prod_DMg.copy()
# DM --> C 50%
df_prod_harvC.loc[:, [*np.arange(2000, 2023)]] =  df_prod_harvC.loc[:, [*np.arange(2000, 2023)]] + df_residue_DMg.loc[:, [*np.arange(2000, 2023)]] * 0.5
df_prod_harvC = df_prod_harvC.groupby(["region", "name_long"], as_index=False)[[*np.arange(2000, 2023)]].sum()

# Slaughter

In [38]:
# load FAO data
df_slaughter_all = pd.read_csv(path_data / "FAO/production/Production_Crops_Livestock_E_All_Data_NOFLAG.csv", encoding_errors="ignore")\
    .query("Element == 'Producing Animals/Slaughtered'")\
    .query("`Area Code`<5000")\

col_sel = ["Area", "Item", "Element", "Unit"]
col_sel = [*col_sel, *[f"Y{year}" for year in range(2000, 2023)]]

df_slaughter_all = df_slaughter_all[col_sel]
sel_idx = df_slaughter_all[[f"Y{year}" for year in range(2000, 2023)]].dropna(how="all").index
df_slaughter_all = df_slaughter_all.loc[sel_idx].reset_index(drop=True)

# Data sepration for countries that became independent after the year 2000
# Sudan (former) --> South Sudan / Sudan
df_sudan, df_sudan_fillna = seperate_data(df_production_all=df_slaughter_all, former='Sudan (former)', new_1='South Sudan', new_2='Sudan')
# Serbia and Montenegro --> Serbia / Montenegro
df_S_M, df_S_M_fillna = seperate_data(df_production_all=df_slaughter_all, former='Serbia and Montenegro', new_1='Serbia', new_2='Montenegro')

In [40]:
dic_liveslaughter = { 'Beef and Buffalo Meat, primary': "Cattle and Buffaloes", 'Buffalo fat, unrendered': "Cattle and Buffaloes", 'Cattle fat, unrendered': "Cattle and Buffaloes", 'Edible offal of buffalo, fresh, chilled or frozen': "Cattle and Buffaloes", 'Edible offal of cattle, fresh, chilled or frozen': "Cattle and Buffaloes", 'Edible offal of goat, fresh, chilled or frozen': "Sheep and Goats", 'Edible offal of pigs, fresh, chilled or frozen': "Swine / pigs", 'Edible offal of sheep, fresh, chilled or frozen': "Sheep and Goats", 'Edible offals of camels and other camelids, fresh, chilled or frozen': "Camels", 'Edible offals of horses and other equines,  fresh, chilled or frozen': "Horses", 'Fat of camels': "Camels", 'Fat of pigs': "Swine / pigs", 'Game meat, fresh, chilled or frozen': "", 'Goat fat, unrendered': "Sheep and Goats", 'Horse meat, fresh or chilled': "Horses", 'Meat of asses, fresh or chilled': "Asses", 'Meat of buffalo, fresh or chilled': "Cattle and Buffaloes", 'Meat of camels, fresh or chilled': "Camels", 'Meat of cattle with the bone, fresh or chilled': "Cattle and Buffaloes", 'Meat of chickens, fresh or chilled': 'Poultry Birds', 'Meat of ducks, fresh or chilled': 'Poultry Birds', 'Meat of geese, fresh or chilled': 'Poultry Birds', 'Meat of goat, fresh or chilled': "Sheep and Goats", 'Meat of mules, fresh or chilled': "Mules and hinnies", 'Meat of other domestic camelids, fresh or chilled': "Camels", 'Meat of other domestic rodents, fresh or chilled': "Other rodents", 'Meat of pig with the bone, fresh or chilled': "Swine / pigs", 'Meat of pigeons and other birds n.e.c., fresh, chilled or frozen': 'Poultry Birds', 'Meat of rabbits and hares, fresh or chilled': "Rabbits and hares", 'Meat of sheep, fresh or chilled': "Sheep and Goats", 'Meat of turkeys, fresh or chilled': 'Poultry Birds', 'Meat, Poultry': 'Poultry Birds', 'Other meat n.e.c. (excluding mammals), fresh, chilled or frozen': "", 'Raw hides and skins of buffaloes': "Cattle and Buffaloes", 'Raw hides and skins of cattle': "Cattle and Buffaloes", 'Raw hides and skins of goats or kids': "Sheep and Goats", 'Raw hides and skins of sheep or lambs': "Sheep and Goats", 'Sheep and Goat Meat': "Sheep and Goats", 'Sheep fat, unrendered': "Sheep and Goats", 'Snails, fresh, chilled, frozen, dried, salted or in brine, except sea snails': ""}

In [41]:
df_slaughter_r1 = pd.concat([
    df_slaughter_all.drop(df_sudan.index).drop(df_S_M.index),
    df_sudan_fillna, df_S_M_fillna
]).reset_index(drop=True)
df_slaughter_r1.columns =  df_slaughter_r1.columns.map(lambda x: int(x[1:]) if x.startswith("Y2") else x)
df_slaughter_r1["name_long"] = df_slaughter_r1["Area"].map(dic_name)
df_slaughter_r1["Unit"] = df_slaughter_r1["Unit"].str.replace(" An", "").str.replace("An", "1").astype(int)
df_slaughter_r1["Item"] = df_slaughter_r1["Item"].map(dic_liveslaughter)
df_slaughter_r1.loc[:, np.arange(2000, 2023)] = df_slaughter_r1.loc[:, np.arange(2000, 2023)].apply(lambda x: x * df_slaughter_r1["Unit"])
df_slaughter_r1 = df_slaughter_r1.groupby(["name_long", "Item"], as_index=False)[np.arange(2000, 2023)].max()
df_slaughter_r1["region"] = df_slaughter_r1["name_long"].map(dict_region)


In [42]:
# Species-specific daily feed intake (kg DM/head/day) --> g DM/head/day
df_livestock_factor = pd.read_excel(path_data / "FAO/yield and stock/Grazing_factor.xlsx", sheet_name="livestock")
df_livestock_factor = df_livestock_factor.melt(id_vars="Unnamed: 0")\
    .rename(columns={"Unnamed: 0": "Item", "variable": "region", "value": "fi"})\
    .assign(fi=lambda _df: _df["fi"]*1000)

In [43]:
df_slaughter_r2 = df_slaughter_r1.merge(df_livestock_factor, on=["region", "Item"], how="left")\
    .dropna(subset="Item")\
    .reset_index(drop=True)

In [44]:
# feed --> DM
# kg DM/head/day
df_slaughter_C = df_slaughter_r2.copy()
df_slaughter_C.loc[:, np.arange(2000, 2023)] = df_slaughter_C.loc[:, np.arange(2000, 2023)].apply(lambda x: x*df_slaughter_C.loc[:, "fi"]*182 )
# DM --> C
df_slaughter_C.loc[:, np.arange(2000, 2023)] = df_slaughter_C.loc[:, np.arange(2000, 2023)].apply(lambda x: x*.5)
df_slaughter_C = df_slaughter_C.groupby("name_long", as_index=True)[[*np.arange(2000, 2023)]].sum()

# stock

In [48]:
df_stock_all = pd.read_csv(path_data / "FAO/production/Production_Crops_Livestock_E_All_Data_NOFLAG.csv", encoding_errors="ignore")\
    .query("Element == 'Stocks'")\
    .query("`Area Code`<5000")\
    # .query("Unit != '1000 No'")

col_sel = ["Area", "Item", "Element", "Unit"]
col_sel = [*col_sel, *[f"Y{year}" for year in range(2000, 2023)]]

df_stock_all = df_stock_all[col_sel]
sel_idx = df_stock_all[[f"Y{year}" for year in range(2000, 2023)]].dropna(how="all").index
df_stock_all = df_stock_all.loc[sel_idx].reset_index(drop=True)


In [49]:
# Sudan (former) --> South Sudan / Sudan
df_sudan, df_sudan_fillna = seperate_data(df_production_all=df_stock_all, former='Sudan (former)', new_1='South Sudan', new_2='Sudan')
# Serbia and Montenegro --> Serbia / Montenegro
df_S_M, df_S_M_fillna = seperate_data(df_production_all=df_stock_all, former='Serbia and Montenegro', new_1='Serbia', new_2='Montenegro')

In [50]:
dic_livestock = {"Cattle and buffaloes": "Cattle and Buffaloes", "Sheep and goats": "Sheep and Goats", "Pigs": "Swine / pigs", "Poultry": "Poultry Birds", "Horses": "Horses", "Asses": "Asses", "Mules": "Mules and hinnies", "Camels": "Camels", "Rabbits": "Rabbits and hares", "Other rodents": "Other rodents", "Other camelids": "Other camelids", 'Bees': "Cattle and Buffaloes", 'Buffalo': "Cattle and Buffaloes", 'Cattle': "Cattle and Buffaloes", 'Chickens': 'Poultry Birds', 'Ducks': 'Poultry Birds', 'Geese': 'Poultry Birds', 'Goats': "Sheep and Goats", 'Sheep': "Sheep and Goats", 'Turkeys': 'Poultry Birds'}

In [51]:
df_stock_r1 = pd.concat([
    df_stock_all.drop(df_sudan.index).drop(df_S_M.index),
    df_sudan_fillna, df_S_M_fillna
]).reset_index(drop=True)
df_stock_r1.columns =  df_stock_r1.columns.map(lambda x: int(x[1:]) if x.startswith("Y2") else x)
df_stock_r1["name_long"] = df_stock_r1["Area"].map(dic_name)
df_stock_r1["region"] = df_stock_r1["name_long"].map(dict_region)
df_stock_r1["Unit"] = df_stock_r1["Unit"].str.replace(" An", "").str.replace("An", "1").str.replace("No", "1").astype(int)
df_stock_r1["Item"] = df_stock_r1["Item"].map(dic_livestock)

In [52]:
# Species-specific daily feed intake (kg DM/head/day) --> g DM/head/day
df_livestock_factor = pd.read_excel(path_data / "FAO/yield and stock/Grazing_factor.xlsx", sheet_name="livestock")
df_livestock_factor = df_livestock_factor.melt(id_vars="Unnamed: 0")\
    .rename(columns={"Unnamed: 0": "Item", "variable": "region", "value": "fi"})\
    .assign(fi=lambda _df: _df["fi"]*1000)

In [ ]:
df_stock_r2 = df_stock_r1.merge(df_livestock_factor, on=["region", "Item"], how="left")\
    .dropna(subset="Item")\
    .reset_index(drop=True)

df_stock_r2.loc[:, np.arange(2000, 2023)] = df_stock_r2.loc[:, np.arange(2000, 2023)].apply(lambda x: x*df_stock_r2.loc[:, "Unit"])
df_stock_r2 = df_stock_r2.drop(columns=["Element", "Unit"]).reset_index(drop=True)

# feed --> DM
df_stock_C = df_stock_r2.copy()
df_stock_C.loc[:, np.arange(2000, 2023)] = df_stock_C.loc[:, np.arange(2000, 2023)].apply(lambda x: x*df_stock_C.loc[:, "fi"]*365)
# DM --> C
df_stock_C.loc[:, np.arange(2000, 2023)] = df_stock_C.loc[:, np.arange(2000, 2023)].apply(lambda x: x*.5)
df_stock_C = df_stock_C.groupby("name_long", as_index=True)[[*np.arange(2000, 2023)]].sum()

# Trade

In [57]:
feed_lst = ['Clover, for forage', 'Forage and silage, alfalfa', 'Forage and silage, maize', 'Gluten feed and meal', 'Vegetable products for feed n.e.c.', 'Compound feed n.e.c.', 'Hay for forage, from legumes', 'Hay for forage, from other crops n.e.c.', 'Hay, non-leguminous', 'Other forage products, n.e.c.', 'Other grasses, for forage', 'Other legumes, for forage', 'Turnips, for forage', 'Beets for fodder', 'Fodder and Feeding Stuff', 'Swedes for fodder', 'Vegetables and roots fodder', 'Bran of barley', 'Bran of buckwheat', 'Bran of cereals n.e.c.', 'Bran of fonio', 'Bran of maize', 'Bran of millet', 'Bran of mixed grain', 'Bran of oats', 'Bran of pulses', 'Bran of rice', 'Bran of rye', 'Bran of sorghum', 'Bran of wheat', 'Feed minerals', 'Feed supplements', 'Feed, compound, cattle', 'Feed, compound, pigs', 'Feed, compound, poultry', 'Feed, food waste preparations', 'Feed, other concentrates nes',]

In [58]:
df_trade_all = pd.read_csv(path_data / "FAO/trade/Trade_CropsLivestock_E_All_Data_NOFLAG.csv", encoding_errors="ignore")\
    .query("`Area Code`<5000")
df_trade_all["name_long"] = df_trade_all["Area"].map(dic_name)

col_sel = ["Area Code", "Area Code (M49)", "Area", "Item", "Element", "Unit", "name_long"]
col_sel = [*col_sel, *[f"Y{year}" for year in range(2000, 2023)]]
df_trade_all = df_trade_all[col_sel]

df_feed_import = df_trade_all.query("Element == 'Import Quantity'").query("Item in @feed_lst")
df_feed_export = df_trade_all.query("Element == 'Export Quantity'").query("Item in @feed_lst")

In [ ]:
df_feed_net_import = df_feed_import.groupby("name_long")[[f"Y{year}" for year in range(2000, 2023)]].sum() - \
    df_feed_export.groupby("name_long")[[f"Y{year}" for year in range(2000, 2023)]].sum()
df_feed_net_import.columns = df_feed_net_import.columns.map(lambda x: int(x.replace("Y", "")))
# Set the moisture content of the feed to 12%
# DM --> C
df_feed_net_import_C = df_feed_net_import.apply(lambda x: x*(1-.12)*0.5)

## residue

In [ ]:
df_used_residues_C = df_residue_DMg.copy()
df_used_residues_C.loc[:, np.arange(2000, 2023)] = df_used_residues_C.loc[:, np.arange(2000, 2023)].apply(lambda x: x*df_used_residues_C["rr"] * 0.5)
df_used_residues_C = df_used_residues_C.groupby("name_long")[np.arange(2000, 2023)].sum()

In [62]:
df_graze_net_harv = df_stock_C\
    .add(df_slaughter_C, fill_value=0)\
    .subtract(df_used_residues_C, fill_value=0)\
    .subtract(df_feed_net_import, fill_value=0)

# wood

In [65]:
wood_lst = ["Roundwood, coniferous", "Roundwood, non-coniferous"]

In [67]:
df_wood_all = pd.read_csv(path_data / "FAO/Forestry/Forestry_E_All_Data_NOFLAG.csv", encoding_errors="ignore")\
    .query("`Area Code`<5000")
df_wood_all["name_long"] = df_wood_all["Area"].map(dic_name)

col_sel = ["Area", "Item", "Element", "Unit", "name_long"]
col_sel = [*col_sel, *[f"Y{year}" for year in range(2000, 2023)]]
df_wood_all = df_wood_all[col_sel]
df_wood_all = df_wood_all.query("Element == 'Production'").query("Item in @wood_lst").drop(columns=["Element"]).fillna(0)
df_wood_all["Item"] = df_wood_all["Item"].str.replace("Roundwood, ", "")
df_wood_all["region"] = df_wood_all["name_long"].map(dict_region)

In [68]:
# Sudan (former) --> South Sudan / Sudan
df_sudan, df_sudan_fillna = seperate_data(df_production_all=df_wood_all, former='Sudan (former)', new_1='South Sudan', new_2='Sudan')
# Serbia and Montenegro --> Serbia / Montenegro
df_S_M, df_S_M_fillna = seperate_data(df_production_all=df_wood_all, former='Serbia and Montenegro', new_1='Serbia', new_2='Montenegro')

df_wood_r1 = pd.concat([
    df_wood_all.drop(df_sudan.index).drop(df_S_M.index),
    df_sudan_fillna, df_S_M_fillna
]).reset_index(drop=True)
df_wood_r1.columns =  df_wood_r1.columns.map(lambda x: int(x[1:]) if x.startswith("Y2") else x)
# df_wood_r1

In [69]:
df_forest_recover_rate = pd.DataFrame([
    ["N. Africa W. Asia", 0.75, 0.75], ["Sub-Saharan Africa", 0.75, 0.81], ["C. Asia", 0.75, 0.69], ["E. Asia", 0.75, 0.69], ["S. Asia", 0.75, 0.69], ["N. America", 0.92, 0.89], ["Latin America", 0.75, 0.84], ["W. Europe", 0.85, 0.86], ["E. Europe", 0.81, 0.79], ["Oceania", 0.92, 0.89]
], columns=["region", "coniferous", "non-coniferous"]).melt(id_vars=["region"])\
    .rename(columns={"variable": "Item", "value": "rec_r"})

In [71]:
# t DM / m^3
# 1e6 g DM / m^3

df_forest_wood_rate = pd.DataFrame([
    ["N. Africa W. Asia", 0.42, 0.57], ["Sub-Saharan Africa", 0.43, 0.57], ["C. Asia", 0.41, 0.57], ["E. Asia", 0.41, 0.57], ["S. Asia", 0.41, 0.57], ["N. America", 0.43, 0.60], ["Latin America", 0.43, 0.60], ["W. Europe", 0.41, 0.57], ["E. Europe", 0.41, 0.57], ["Oceania", 0.41, 0.57]
], columns=["region", "coniferous", "non-coniferous"]).melt(id_vars=["region"])\
    .rename(columns={"variable": "Item", "value": "wood_r"})

In [72]:
df_wood_DMg = df_wood_r1.merge(df_forest_recover_rate, on=["region", "Item"], how="left").merge(df_forest_wood_rate, on=["region", "Item"], how="left")
# 1e6 g DM / m^3
df_wood_DMg.loc[:, np.arange(2000, 2023)] = df_wood_DMg.loc[:, np.arange(2000, 2023)].apply(lambda x: x / df_wood_DMg["rec_r"] * df_wood_DMg["wood_r"] * 1000000).fillna(0)
df_wood_DMg = df_wood_DMg.groupby(["region", "name_long"], as_index=False)[[*np.arange(2000, 2023)]].sum()

# DM --> C
df_wood_C = df_wood_DMg.copy()
df_wood_C.loc[:, np.arange(2000, 2023)] = df_wood_C.loc[:, np.arange(2000, 2023)].apply(lambda x: x *0.5)

# Harv

In [95]:
df_harv_total = fv.fillna(0).add(\
    df_prod_harvC.set_index("name_long").drop(columns=["region", "area"]).fillna(0), fill_value=0).add(\
    df_wood_C.set_index("name_long").drop(columns=["region", "area"]).fillna(0), fill_value=0)
df_harv_total.to_csv(path_data / "zonalstats/harv_all_total.csv")